In [2]:
import pandas
import cv2
import os
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
import scipy


In [3]:
PATH_TO_ORIGINAL_IMAGE_DIRECTORY = "../CharactersPlate"
THRESHOLD = 127
WINDOW_SIZE = 1000

In [30]:
def rename_files(dir_path):
    for filename in os.listdir(dir_path):
        full_path = os.path.join(dir_path, filename)
        name, extension = os.path.splitext(filename)
        new_filename = name[-1] + extension
        new_path = os.path.join(dir_path, new_filename)
        os.rename(full_path, new_path)

#rename_files(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)

In [4]:
def show_image_on_window(image: np.array, window_name: str = "Image") -> None:
    scale = max(image.shape[:2]) / WINDOW_SIZE
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, int(
        image.shape[1] // scale), int(image.shape[0] // scale))
    cv2.imshow(window_name, image)
    cv2.waitKey(3000)
    cv2.destroyWindow(window_name)

In [22]:
def adapt_image(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image_binary = cv2.threshold(image, THRESHOLD, 255, cv2.THRESH_BINARY)
    return image_binary/255

In [23]:
def load_images(dir_path):
        images = []
        chars = []
        filenames = []
        for file_name in os.listdir(dir_path):
                name, extension = os.path.splitext(file_name)
                if extension == ".png":
                        original_image = cv2.imread(dir_path + "/" + file_name)
                        binary_image = adapt_image(original_image)
                        images.append(binary_image)
                        chars.append(name)
                        filenames.append(file_name)
        return np.array(images), np.array(chars), filenames

DATA AUGMENTATION

In [34]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.05,
    fill_mode='nearest'
)

In [23]:
X = np.expand_dims(X, axis=-1)

datagen.fit(X)

In [24]:
def gen_aug_images(X, y, dir_path, num_aug_img=100):
    tag_count = {}
    
    for image, tag in zip(X, y):
        tag_str = str(tag)
        if tag_str not in tag_count:
            tag_count[tag_str] = 0
        image = np.expand_dims(image, axis=0)
        i = 0
        for batch in datagen.flow(image, batch_size=1):
            tag_count[tag_str] += 1
            filename = f"{tag_str}_{tag_count[tag_str]}.png"
            full_path = os.path.join(dir_path, filename)
            bin_img = (batch[0] > 0.5).astype(np.uint8) * 255
            cv2.imwrite(full_path, bin_img)
            i += 1
            if i >= num_aug_img:
                break 


gen_aug_images(X, Y, PATH_TO_ORIGINAL_IMAGE_DIRECTORY)


### SVM MODEL ###

In [60]:
def load_data_svm(dir_path):
    images = []
    chars = []
    for file_name in os.listdir(dir_path):
        name, extension = os.path.splitext(file_name)
        if extension == ".png":
            original_image = cv2.imread(dir_path + "/" + file_name)
            binary_image = adapt_image(original_image)
            flat_image = binary_image.flatten()
            images.append(flat_image)
            chars.append(name[0])
    return np.array(images), np.array(chars)

In [61]:
X, y = load_data_svm(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)

In [63]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svm_model = SVC(kernel='rbf') 
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 99.82%


ELIMINAR ARCHIVOS

In [15]:
def eliminar_png_largos(direccion_carpeta):
    # Recorre todos los archivos en la carpeta dada
    for archivo in os.listdir(direccion_carpeta):
        ruta_completa = os.path.join(direccion_carpeta, archivo)
        
        # Verifica si es un archivo (no directorio)
        if os.path.isfile(ruta_completa):
            # Obtiene el nombre del archivo sin la extensión y su extensión
            nombre_archivo, extension = os.path.splitext(archivo)
            
            # Comprueba si la extensión es '.png' y si el nombre tiene 7 o más caracteres
            if extension == '.png' and len(nombre_archivo) >= 3:
                # Elimina el archivo
                os.remove(ruta_completa)

eliminar_png_largos(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)
